<a href="https://colab.research.google.com/github/davidraamirez/GradientWithoutBackpropagation/blob/main/LogisticRegression_fwd_gradient.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import tensorflow_datasets as tfds
from sklearn.model_selection import train_test_split

In [2]:
%pip install torchmetrics --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.2/517.2 KB 3.8 MB/s eta 0:00:00


In [3]:
import torchmetrics
import torchvision

Loading and preprocessing the data

In [4]:
#Load the dataset
penguins = tfds.load('penguins', as_supervised=True, split='train')

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/334 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/penguins/processed/1.0.0.incompleteOCDH9A/penguins-train.tfrecord*...:   0…

Dataset penguins downloaded and prepared to /root/tensorflow_datasets/penguins/processed/1.0.0. Subsequent calls will reuse this data.


In [5]:
# By default, the Dataset object is an iterator over the elements.
# The instructions below extract the underlying tensors.
X, y = penguins.batch(500).get_single_element()
X, y = X.numpy(), y.numpy()

In [6]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, stratify=y)

In [7]:
Xtrain = torch.from_numpy(Xtrain).float()
Xtest = torch.from_numpy(Xtest).float()

In [8]:
ytrain = torch.from_numpy(ytrain).long()
ytest = torch.from_numpy(ytest).long()

Define Logistic Regression

In [9]:
from torch import nn

In [10]:
class SimpleLogisticRegression(nn.Module):
  def __init__(self, input_size, w, b):
    super().__init__()
    self.weight = nn.Parameter(w)
    self.bias = nn.Parameter(b)

  def forward(self, x):
    x = x.reshape(1, -1)
    return torch.softmax(x@self.weight + self.bias, 1)

In [11]:
# We check if CUDA is available. If you do not see it,
# activate a GPU from Runtime >> Change runtime type and 
# restart the notebook.
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


Initialize the parameters

In [12]:
# We initialize the parameters randomly and the model with an input size
w = torch.randn((4, 3), requires_grad=False)
b = torch.randn((3, ), requires_grad=False)
LG = SimpleLogisticRegression(4, w, b).to(device)

In [13]:
# Note: we also need to move data when asking for a prediction
print(LG(Xtrain[0].to(device)))

tensor([[0.0940, 0.2397, 0.6663]], device='cuda:0', grad_fn=<SoftmaxBackward0>)


Train and evaluate the network with forward gradient

In [14]:
def accuracy(ytrue, ypred):
  return (ypred.argmax(1) == ytrue).float().mean()

In [17]:
# Average accuracy at initialization is 10% (random guessing).
accuracy(ytrain[0].to(device), LG(Xtrain[0].to(device)))

tensor(0., device='cuda:0')

Define Cross Entropy

In [18]:
def cross_entropyW(w, ytrue, x, b):
  """ Cross-entropy loss.
  Inputs:
  - ytrue (n,): vector of indices for the correct class.
  - ypred (n, 3): predictions of the model.
  Returns the average cross-entropy.
  """
  # This is called integer array indexing in NumPy:
  # https://numpy.org/doc/stable/user/basics.indexing.html#integer-array-indexing
  ypred=torch.randn((ytrue.shape[0],3))
  for j in range (ytrue.shape[0]):
    xj = x[j].reshape(1, -1)
    ypred[j]=torch.softmax(xj@w+b,1)
  return - ypred[torch.arange(0, ypred.shape[0]), ytrue].log().mean()

In [20]:
def cross_entropyB(b, ytrue, x, w):
  """ Cross-entropy loss.
  Inputs:
  - ytrue (n,): vector of indices for the correct class.
  - ypred (n, 3): predictions of the model.
  Returns the average cross-entropy.
  """
  # This is called integer array indexing in NumPy:
  # https://numpy.org/doc/stable/user/basics.indexing.html#integer-array-indexing
  ypred=torch.randn((ytrue.shape[0],3))
  for j in range (ytrue.shape[0]):
    xj = x[j].reshape(1, -1)
    ypred[j]=torch.softmax(xj@w+b,1)
  return - ypred[torch.arange(0, ypred.shape[0]), ytrue].log().mean()

In [23]:
print(cross_entropyB(b, ytrain,Xtrain, w))
print(cross_entropyW(w,ytrain,Xtrain,b))

tensor(2.0122)
tensor(2.0122)


In [25]:
def beale_function(x):
  return (torch.pow(torch.tensor([1.5])-x[0]+x[0]*x[1],2) + torch.pow(torch.tensor([2.25])-x[0]+x[0]*torch.pow(x[1],2),2)+torch.pow(torch.tensor([2.625])-x[0]+x[0]*torch.pow(x[1],3),2))

In [26]:
def rosenbrock_function(x):
  sum=0
  for i in range (x.size(0) -1):
    sum += (100*torch.pow(x[i+1] - torch.pow(x[i], 2), 2) + torch.pow(1-x[i], 2))
  return sum

In [27]:
def sphere_function(x):
  sum=0
  for i in range(x.size(0)):
    sum += torch.pow(x[i], 2)
  return sum

In [29]:
from functorch import jvp
from functools import partial
import time 

In [47]:
def train_fwd_gradient(x, y):

  #x, y = x.to(device), y.to(device)

  l_rate0 = 0.2

  #Parameters
  w = torch.randn((4, 3))
  b = torch.randn((3, ))

  loss = cross_entropyW(w, y, x, b)

  LG = SimpleLogisticRegression(4, w, b)

  t=0
  t0=time.time()
  print('Iteracion', t, 'loss', loss)

  while (loss>0.1) :

    vw=torch.randn(w.shape)
    vw = (vw - torch.mean(vw))/torch.std(vw)
    vb=torch.randn(b.shape)
    vb = (vb - torch.mean(vb))/torch.std(vb)

    fw = partial(cross_entropyW,ytrue=y, x=x, b=b)
    fb = partial(cross_entropyB,ytrue=y, x=x, w=w)

    loss, dtw = jvp(fw,(w, ), (vw, ))
    ftb, dtb = jvp(fb,(b, ), (vb, ))

    gtw = vw*dtw
    gtb = vb*dtb

    w = w - l_rate0*gtw
    b = b - l_rate0*gtb
    t1=time.time()
    t+=t1-t0
    t0=t1
    print('Time', t, 'loss', loss)
  return w, b

In [48]:
w, b = train_fwd_gradient(Xtrain, ytrain)

Iteracion 0 loss tensor(2.2132)
Time 0.08210968971252441 loss tensor(2.2132)
Time 0.1608743667602539 loss tensor(2.2124)
Time 0.24217510223388672 loss tensor(2.1150)
Time 0.3203601837158203 loss tensor(2.0315)
Time 0.39556288719177246 loss tensor(2.0154)
Time 0.48374080657958984 loss tensor(1.9405)
Time 0.5594832897186279 loss tensor(1.8550)
Time 0.6387946605682373 loss tensor(1.8519)
Time 0.7190721035003662 loss tensor(1.8346)
Time 0.804671049118042 loss tensor(1.8260)
Time 0.888045072555542 loss tensor(1.7178)
Time 0.9759786128997803 loss tensor(1.6717)
Time 1.0530319213867188 loss tensor(1.6371)
Time 1.1314730644226074 loss tensor(1.5411)
Time 1.2157061100006104 loss tensor(1.5154)
Time 1.2910146713256836 loss tensor(1.4746)
Time 1.3676481246948242 loss tensor(1.4629)
Time 1.4472289085388184 loss tensor(1.4287)
Time 1.5253684520721436 loss tensor(1.4015)
Time 1.5999202728271484 loss tensor(1.3808)
Time 1.6800384521484375 loss tensor(1.3623)
Time 1.7573518753051758 loss tensor(1.3231

In [49]:
LG = SimpleLogisticRegression(4, w, b)
ypred=torch.randn(Xtest.size(0),3)
error=0
for i in range (Xtest.size(0)):
  ypred[i]=LG(Xtest[i])
  if (LG(Xtest[i]).argmax(1)- ytest[i])!=0:
    error = error+ 1
error= error / Xtest.size(0)


In [50]:
print(error)

0.011904761904761904


In [51]:
print(accuracy(ytest,ypred))

tensor(0.9881)
